In [ ]:
import os
from sql_nameguard.llm_suggest import LLMSuggester
from sql_nameguard.analyze import SQLAnalyzer

from dotenv import load_dotenv
load_dotenv()

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
sample_sql = """
WITH customer_order_totals AS (          -- ✅ GOOD CTE name
    SELECT
        cust.customer_id,               -- ✅ clear column name
        SUM(item.quantity * item.unit_price) AS lifetime_value  -- ✅ good semantic alias
    FROM customers AS cust              -- ✅ good table alias
    JOIN orders AS ord                  -- ✅ good table alias
        ON cust.customer_id = ord.customer_id
    JOIN order_items AS item            -- ✅ good table alias
        ON ord.order_id = item.order_id
    WHERE ord.status = 'COMPLETED'
    GROUP BY cust.customer_id
),

t2 AS (                                 -- ❌ POOR CTE name (what is t2?)
    SELECT
        cot.customer_id AS cid,         -- 😐 meh alias: shorter but less clear
        cot.lifetime_value AS v,        -- ❌ bad alias: single letter, opaque meaning
        COUNT(o2.order_id) AS cnt       -- ❌ vague alias, "cnt" of what?
    FROM customer_order_totals AS cot   -- ✅ good table alias
    JOIN orders AS o2                   -- 😐 okay but a bit cryptic if more tables existed
        ON cot.customer_id = o2.customer_id
    WHERE o2.status = 'COMPLETED'
    GROUP BY cot.customer_id, cot.lifetime_value
),

high_value_customers AS (               -- ✅ GOOD CTE name
    SELECT
        cid AS customer_id,             -- 😐 fixing cid… but alias hop is unnecessary
        v   AS lifetime_value,          -- ❌ still opaque → better to keep original name
        cnt AS completed_order_count    -- ✅ finally a descriptive alias
    FROM t2
    WHERE v > 1000
)

SELECT
    hvc.customer_id,                    -- ✅ clear
    hvc.lifetime_value,                 -- ✅ clear
    hvc.completed_order_count,          -- ✅ clear
    CONCAT(cust.first_name, ' ', cust.last_name) AS full_name,  -- ✅ good column alias
    cust.email,
    cust.city AS customer_city          -- ✅ descriptive alias
FROM high_value_customers AS hvc        -- ✅ good table alias
JOIN customers AS cust                  -- ✅ good table alias
    ON cust.customer_id = hvc.customer_id
ORDER BY hvc.lifetime_value DESC;

"""

In [ ]:
from sql_nameguard.parse_sql import SQLParser

In [ ]:
# initial_aliases = SQLParser.parse(sample_sql)

In [ ]:
# initial_aliases

In [ ]:
analyzer = SQLAnalyzer()

aliases = analyzer.analyze(sample_sql, threshold=0.5)

In [ ]:
aliases

In [ ]:
suggester = LLMSuggester(model='openai/gpt-5-nano', api_key=openai_api_key)

In [ ]:
responses = []
for alias in aliases:
    if alias['similarity'] < 0.4:
        print(f'Selecting alternates for alias: {alias["alias"]}...')
        response = suggester.suggest_aliases(alias_type=alias['type'], code=alias['code'])
        responses.append(response)

In [ ]:
responses